In [25]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, merge, Dense, TimeDistributed, Dropout
from keras.layers import Bidirectional, concatenate, SpatialDropout1D
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from numpy import dot


import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import add

data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [26]:
data = data.fillna(method="ffill")
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [27]:
wordwa = list(set(data["Word"].values))
words = []

for w in wordwa :
    w = str(w)
    w = w.lower()
    words.append(w)
print(words[0:10])
n_words = len(words); 
n_words

['particularly', 'scan', 'minnesota', 'yuanshi', 'mastered', 'smearing', '63.95', 'tkachev', 'roseneft', 'abduction']


35178

In [28]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

72


In [29]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [30]:
print(char2idx['k'])
print(char2idx['-'])

58
9


In [31]:
max_word_length = 25
embedded_char_vector_length = 74 # Length of character dictionary
char_feature_output = 10

In [32]:
X_char = []
for word in words :
    l1 = []
    #if (len(word) > max_word_length) :
    #    print(len(word))
    for i in range(max_word_length):
        try:
            l1.append(char2idx.get(word[i]))
        except:
            l1.append(char2idx.get("PAD"))    
    
    X_char.append(l1)

In [33]:
print(X_char[0:5])

[[59, 38, 65, 31, 17, 61, 20, 63, 38, 65, 63, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [13, 61, 38, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6, 17, 18, 18, 28, 13, 26, 31, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 20, 38, 18, 13, 15, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6, 38, 13, 31, 28, 65, 28, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [34]:
char_input = Input(shape=(max_word_length,), dtype='float32', name='char_input')
char_input1 = Embedding(embedded_char_vector_length, 16, input_length = max_word_length)(char_input)
char_input2 = Dropout(0.2)(char_input1)
'''lstm_out_forward = LSTM(char_feature_output, dropout=0.2, recurrent_dropout=0.2)(char_input2)
lstm_out_backward = LSTM(char_feature_output, go_backwards=True, dropout=0.2, recurrent_dropout=0.2)(char_input2)'''

merged = (Bidirectional(LSTM(units=5, return_sequences=False,trainable = False,
                                recurrent_dropout=0.5)))(char_input2)
#merged = merge([lstm_out_forward, lstm_out_backward], mode='concat', concat_axis=1)

#merged = add([lstm_out_forward, lstm_out_backward])
#model = Model(input=[char_input], output=[merged])
model = Model(char_input, merged)

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_input (InputLayer)      (None, 25)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 25, 16)            1184      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25, 16)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10)                880       
Total params: 2,064
Trainable params: 2,064
Non-trainable params: 0
_________________________________________________________________


In [40]:
p = model.layers[1].get_weights()

print(np.array(p).shape)
print(p)

(1, 74, 16)
[array([[-0.04473716,  0.0233576 , -0.01393676, ..., -0.01707486,
         0.03105256,  0.01986625],
       [ 0.04827   ,  0.04199752, -0.0108634 , ...,  0.03234868,
         0.01352074, -0.03208601],
       [-0.04808122, -0.02279522,  0.00911144, ...,  0.02500156,
         0.0486672 ,  0.00822457],
       ...,
       [-0.01482625, -0.04989877, -0.03241383, ...,  0.00977838,
         0.01799469,  0.02689276],
       [ 0.02179087,  0.03421267,  0.01698203, ...,  0.01290497,
         0.00871219, -0.04210969],
       [ 0.02940394, -0.04481616,  0.04241074, ..., -0.03438262,
        -0.03603367,  0.01527869]], dtype=float32)]


In [42]:
X1 = np.array(X_char[0:5])
print(X1)
train_char_vectors = model.predict(X1)
        

[[59 38 65 31 17 61 20 63 38 65 63  5  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [13 61 38 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [ 6 17 18 18 28 13 26 31 38  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [ 5 20 38 18 13 15 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]
 [ 6 38 13 31 28 65 28 34  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0]]


In [43]:
print(train_char_vectors.shape)
print("\n")
print(train_char_vectors)

(5, 10)


[[-0.0013796  -0.05276449  0.00605738 -0.01108203 -0.033244   -0.00124549
   0.00733913 -0.00580821  0.00102096  0.00264938]
 [-0.00027438 -0.05564112  0.00448809 -0.01132372 -0.03460157  0.01024884
   0.0002259   0.0138014   0.00765138  0.01188171]
 [-0.00094933 -0.0539206   0.00543669 -0.01118491 -0.03378528 -0.00419328
   0.00699057  0.00928531  0.00726668  0.0041477 ]
 [-0.00012118 -0.05572236  0.00451243 -0.01142558 -0.03442071  0.0116543
  -0.00252125 -0.00296156 -0.00438434  0.00400033]
 [-0.00065556 -0.05451943  0.00513354 -0.01129829 -0.03394692  0.00228444
   0.01018188  0.0002169   0.0099055  -0.00106856]]


In [44]:
def pred_new_num_enc(s) :
    
    pred = s

    l = []
    ip = []
    k=0

    for i in range(len(pred)):
        l.append(char2idx.get(pred[i]))
    #print(l)
    #print(char2idx.get("PAD"))        


    length = len(l)
    #print(length)

    if length < max_word_length :
        k = max_word_length - length
        print(k)

        for i in range (0,k) :
            l.append(char2idx.get("PAD"))    

    else :
        l = l[:max_word_length]
    #ip.append(l)

    l = np.array([l])
    #print(l)
    
    return l

In [45]:
def pred_new_vec() :
    train_char_vectors = model.predict(l)

    #print(train_char_vectors)
    
    return train_char_vectors

In [46]:
a1 = pred_new_num_enc("name")
a2 = pred_new_num_enc("nam")

print(cosine_similarity(a1 , a2))

21
22
[[0.83490321]]


In [47]:
a3 = pred_new_num_enc("name")
a4 = pred_new_num_enc("nam-e")

print(cosine_similarity(a3 , a4))

21
20
[[0.78228803]]


In [48]:
a5 = pred_new_num_enc("address")
a6 = pred_new_num_enc("addr")

print(cosine_similarity(a5 , a6))

18
21
[[0.93634605]]


In [49]:
a7 = pred_new_num_enc("address")
a8 = pred_new_num_enc("adres")

print(cosine_similarity(a7 , a8))

18
20
[[0.83118823]]


In [50]:
a9 = pred_new_num_enc("contact")
a10 = pred_new_num_enc("phone-no")

print(cosine_similarity(a9 , a10))

18
17
[[0.81579926]]
